# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Uninstall flash-attention (the PyPI package name may vary)
!pip uninstall flash-attn -y
!pip uninstall xformers -y

!pip install flash-attn
!pip install xformers


# Clean up the notebook
clean_notebook()



Notebook cleaned.


In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

# Sign in to Hugging Face

1. If you haven't already done so, create a free HuggingFace account at https://huggingface.co and navigate to Settings, then Create a new API token, giving yourself write permissions

2. Press the "key" icon on the side panel to the left, and add a new secret:
`HF_TOKEN = your_token`

3. Execute the cell below to log in.

In [3]:
import os
from huggingface_hub import login

hf_token = os.environ.get('HF_TOKEN')

# Log into Hugging Face, adding credentials to git if needed
login(hf_token, add_to_git_credential=False)

In [4]:
# instruct models

LLAMA    = "meta-llama/Meta-Llama-3.1-8B-Instruct"
Deepseek ="deepseek-ai/DeepSeek-R1-Distill-Llama-8B"


In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant answer in only thai language"},
    {"role": "user", "content": "เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า"}
  ]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole family of models.

In [6]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

If the next cell gives you a 403 permissions error, then please check:
1. Are you logged in to HuggingFace? Try running `login()` to check your key works
2. Did you set up your API key with full read and write permissions?
3. If you visit the Llama3.1 page at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B, does it show that you have access to the model near the top?


In [7]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

In [8]:
# The model
model = AutoModelForCausalLM.from_pretrained(
    LLAMA, 
    device_map="auto", 
    quantization_config=quant_config,
    
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,591.5 MB


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [11]:
outputs = model.generate(inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant answer in only thai language<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<|eot_id|><|start_header_id|>assistant

นิทานเรื่อง "แมว สุนัข และ เต่า"

มีแมว สุนัข และ เต่า 3 ตัว ที่อยู่ในสวนใหญ่

แมวเป็นแมวที่ฉลาดและสามารถวิ่งได้เร็วมาก สุนัขเป็นสุนัขที่มีใจดีและสามารถช่วยเหลือได้ และเต่าเป็นเต่าที่สามารถปีนป่ายได้ยอดเยี่ยม

วันหนึ่ง พวกมันก็พบกับปัญหาที่เหมือนกัน นั่นคือ พวกมันไม่มีน้ำดื่ม

แมวพูดว่า "ฉันสามารถวิ่งไปหาน้ำได้" แต่สุนัขพูดว่า "ฉันไม่สามารถวิ่งไปหาน้ำได้ แต่ฉันสามารถช่วยเหลือคุณได้"

เตาพูดว่า "ฉันสามารถปีนป่ายไปหาน้ำได้ แต่ฉันไม่สามารถแบ่งปันน้ำกับคุณได้"

แมวและสุนัขได้ขอเตาให้แบ่งปันน้ำกัน แต่เตาปฏิเสธ

สุนัขและแมวได้ใช้ความฉลาดและความสามารถของพวกเขาเพื่อไปหาน้ำและแบ่งปันกัน

สุดท้าย พวกมันก็ได้น้ำและแบ่งปันกัน และพวกมันก็ได้เรียนรู้ว่า การช่วยเหลือกันและแบ่งปั

In [12]:
# Clean up

del inputs, outputs, model
torch.cuda.empty_cache()

## A couple of quick notes on the next block of code:

I'm using a HuggingFace utility called TextStreamer so that results stream back.
To stream results, we simply replace:  
`outputs = model.generate(inputs, max_new_tokens=80)`  
With:  
`streamer = TextStreamer(tokenizer)`  
`outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)`

Also I've added the argument `add_generation_prompt=True` to my call to create the Chat template. This ensures that Phi generates a response to the question, instead of just predicting how the user prompt continues. Try experimenting with setting this to False to see what happens. You can read about this argument here:

https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts

Thank you to student Piotr B for raising the issue!

In [13]:
# Wrapping everything in a function - and adding Streaming and generation prompts
def generate(model, messages):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Prepare inputs
    inputs = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt", 
        add_generation_prompt=True
    ).to("cuda")
    
    # Create a streamer for live output (optional)
    streamer = TextStreamer(tokenizer)
    
    # Load the model
    model = AutoModelForCausalLM.from_pretrained(
        model, 
        device_map="auto", 
        quantization_config=quant_config
    )
    
    # Generate with temperature=0 and additional parameters
    outputs = model.generate(
        inputs,
        max_new_tokens=3000,
        temperature=0.0,       # Deterministic behavior
        do_sample=False,       # Typically set False if temperature=0
        top_p=1.0,             # No nucleus sampling constraint
        top_k=50,              # You can adjust this if needed
        repetition_penalty=1.0, # Adjust to penalize repeated phrases
        streamer=streamer      # Remove if you don't need live output
    )
    
    # Clean up
    del tokenizer, streamer, model, inputs, outputs
    torch.cuda.empty_cache()


In [14]:
generate(LLAMA, messages)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant answer in only thai language<|eot_id|><|start_header_id|>user<|end_header_id|>

เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<|eot_id|><|start_header_id|>assistant<|end_header_id|>

มีครั้งหนึ่ง มีแมว สุนัข และ เต่าอยู่ในสวนร่วมกัน แมวชอบนอนพักผ่อน สุนัขชอบเล่น และ เต่าชอบว่ายน้ำ

วันหนึ่ง แมว สุนัข และ เต่าได้พบกันในสวน แมวพูดว่า "ฉันเหนื่อยมาก ฉันอยากนอนพักผ่อน" สุนัขพูดว่า "ฉันอยากเล่น ฉันอยากเล่นกับเพื่อนๆ" เต่าพูดว่า "ฉันอยากว่ายน้ำ ฉันอยากว่ายน้ำในบ่อ"

แมว สุนัข และ เต่าได้พูดคุยกันและตัดสินใจว่าจะไปว่ายน้ำในบ่อพร้อมกัน แมวจะนอนพักผ่อนบนชายหาด สุนัขจะเล่นกับเพื่อนๆ และ เต่าจะว่ายน้ำในบ่อ

เมื่อถึงเวลาว่ายน้ำ เต่าได้ว่ายน้ำไปในบ่อ สุนัขได้เล่นกับเพื่อนๆ และ แมวได้นอนพักผ่อนบนชายหาด

หลังจากว่ายน้ำไปหนึ่งรอบ เต่าได้กลับมาและพูดว่า "ฉันเหนื่อยมาก ฉันอยากนอนพักผ่อน" สุนัขพูดว่า "ฉันเหนื่อยมาก ฉันอยากนอนพักผ่อน" แมวพูดว่า "

In [15]:
generate(Deepseek, messages)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<｜begin▁of▁sentence｜>You are a helpful assistant answer in only thai language<｜User｜>เล่านิทานให้ที in thai  เกี่ยวกับ แมว และ สุนัข และ เต่า<｜Assistant｜><think>
Okay, I need to create a story in Thai about a cat, a dog, and a turtle. The user wants it to be engaging and suitable for children. Let me think about how to structure this.

First, I should come up with a title that's catchy and relates to the three characters. Maybe something like "Three Friends: The Tale of Miao, Huan, and Pa." That sounds fun and inclusive.

Next, I'll introduce each character with distinct traits. Miao the cat can be curious and clever, Huan the dog loyal and brave, and Pa the turtle wise and calm. This setup allows each to have unique roles and interactions.

The setting should be a village near a river where they live together. This gives a sense of community and provides a backdrop for their adventures. I'll mention their daily lives, showing how they get along despite being different animals.

Now, t